In [125]:
import wbgapi as wb
import pandas as pd
import statistics as stats

In [80]:
country_map = wb.economy.DataFrame()

In [81]:
country_map.reset_index(inplace=True)

country_map.head()

,id,name,aggregate,longitude,latitude,region,adminregion,lendingType,incomeLevel,capitalCity
0,ABW,Aruba,False,-70.0167,12.51670,LCN,,LNX,HIC,Oranjestad
1,AFE,Africa Eastern and Southern,True,NaN,NaN,,,,,
2,AFG,Afghanistan,False,69.1761,34.52280,MEA,MNA,IDX,LIC,Kabul
3,AFW,Africa Western and Central,True,NaN,NaN,,,,,
4,AGO,Angola,False,13.2420,-8.81155,SSF,SSA,IBD,LMC,Luanda


In [82]:
country_map = country_map[['id','name','incomeLevel']]
country_map.head()

,id,name,incomeLevel
0,ABW,Aruba,HIC
1,AFE,Africa Eastern and Southern,
2,AFG,Afghanistan,LIC
3,AFW,Africa Western and Central,
4,AGO,Angola,LMC


In [ ]:
# lookup for the indicators available in the library
wb.series.info()

In [121]:
selected_indicators = ['NY.GDP.MKTP.CD', 'ST.INT.RCPT.CD', 'ST.INT.ARVL','SP.POP.TOTL']
selected_countries = ['USA', 'MEX', 'FRA', 'ITA', 'JPN', 'THA']

In [160]:
data = wb.data.DataFrame(
    selected_indicators,
    selected_countries,
    time=range(2000, 2021)
)

In [186]:
data.index

MultiIndex([('FRA', 'NY.GDP.MKTP.CD'),
            ('FRA',    'SP.POP.TOTL'),
            ('FRA',    'ST.INT.ARVL'),
            ('FRA', 'ST.INT.RCPT.CD'),
            ('ITA', 'NY.GDP.MKTP.CD'),
            ('ITA',    'SP.POP.TOTL'),
            ('ITA',    'ST.INT.ARVL'),
            ('ITA', 'ST.INT.RCPT.CD'),
            ('JPN', 'NY.GDP.MKTP.CD'),
            ('JPN',    'SP.POP.TOTL'),
            ('JPN',    'ST.INT.ARVL'),
            ('JPN', 'ST.INT.RCPT.CD'),
            ('MEX', 'NY.GDP.MKTP.CD'),
            ('MEX',    'SP.POP.TOTL'),
            ('MEX',    'ST.INT.ARVL'),
            ('MEX', 'ST.INT.RCPT.CD'),
            ('THA', 'NY.GDP.MKTP.CD'),
            ('THA',    'SP.POP.TOTL'),
            ('THA',    'ST.INT.ARVL'),
            ('THA', 'ST.INT.RCPT.CD'),
            ('USA', 'NY.GDP.MKTP.CD'),
            ('USA',    'SP.POP.TOTL'),
            ('USA',    'ST.INT.ARVL'),
            ('USA', 'ST.INT.RCPT.CD')],
           names=['economy', 'series'])

In [213]:
data.loc[[('FRA','ST.INT.ARVL')]].mean(axis=1).values[0]

np.float64(194400352.94117647)

In [232]:
data.loc[[('FRA','ST.INT.RCPT.CD')]].mean(axis=1).values[0]

np.float64(57693761904.7619)

In [236]:
type(data.loc[[('FRA','ST.INT.RCPT.CD')]])

pandas.core.frame.DataFrame

In [247]:
data.loc[[('FRA','ST.INT.ARVL')]]["YR2000"].isna()

economy  series     
FRA      ST.INT.ARVL    True
Name: YR2000, dtype: bool

In [267]:
def ratiofill(fill_df, ref_df):
    """
    fill missing values in fill_df using the ratio of ref_df
    The ratio is calculated as the mean of the values in ref_df
    for each year, and then applied to fill_df.
    """
    fill_df = fill_df.copy()

    # find the ratio of the reference dataframe
    ratio = fill_df.mean(axis=1).values[0]/ref_df.mean(axis=1).values[0]

    # fill the NaN values
    for column in fill_df.columns:
        if fill_df[column].isna().values[0]:
            print(f"looking on {column} and found that the na check is {fill_df[column].isna().values[0]}")
            # print(fill_df[column])
            fill_df[column].values[0] = ref_df[column] * ratio

    return fill_df

In [268]:
ratiofill(data.loc[[('FRA','ST.INT.ARVL')]], data.loc[[('FRA','ST.INT.RCPT.CD')]])

looking on YR2000 and found that the na check is True
looking on YR2001 and found that the na check is True
looking on YR2002 and found that the na check is True
looking on YR2003 and found that the na check is True


C:\Users\abejr\AppData\Local\Temp\ipykernel_24908\91217558.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  fill_df[column].values[0] = ref_df[column] * ratio


,,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,YR2009,...,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020
economy,series,,,,,,,,,,,,,,,,,,,,,
FRA,ST.INT.ARVL,1.304207e+08,1.274555e+08,1.385311e+08,1.569388e+08,190282000.0,185829000.0,193882000.0,193319000.0,193571000.0,192369000.0,...,196595000.0,197522000.0,204410000.0,206599000.0,203302000.0,203042000.0,207274000.0,211998000.0,217877000.0,117109000.0


In [238]:
data.loc[[('FRA','ST.INT.ARVL')]].isna()

,,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,YR2009,...,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020
economy,series,,,,,,,,,,,,,,,,,,,,,
FRA,ST.INT.ARVL,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# # join the country with income levels
# data = data.merge(country_map, left_on='economy', right_on='id')
# data[data['economy']=='FRA']